In [ ]:
from solcx import set_solc_version, compile_files
from web3 import Web3

from eth_tester import EthereumTester

In [ ]:
set_solc_version('v0.5.0')

In [ ]:
# use eth-tester for blockchain
# deploy contract using mock backend

In [ ]:
# Mockbackend set
TESTER = EthereumTester()

In [ ]:
# construct path to contract file
def get_contract_path(contract_name):
    return "./contracts/"+contract_name+".sol"

In [ ]:
# compile contract and get the result of compilation
def compile_contract(contract_name):
    # Solidity Compiler
    source_file_name = get_contract_path(contract_name)
    compiled_sol = compile_files([source_file_name]) # Compiled source code
    return compiled_sol[source_file_name + ":" + contract_name]

In [ ]:
# get a web3 object instance, paramaterized with the contract
def get_w3(provider, account):
    # web3.py instance
    w3 = Web3(provider)
    
    # set account as default sender of transactions
    w3.eth.defaultAccount = account
    
    # returns instance of web3 object .. interacts with the blockchain
    return w3

In [ ]:
# Deploy the contract to the blockchain
# takes in the w3 instance which is initialized with a provider and default account
# *c_args takes in parameters from any constructors in the contract
def deploy_contract(w3, compiled_contract, *c_args):
    # get the contract interface from compiled contract code; provides the contract functions, constructor if exists
    contract_interface = w3.eth.contract(abi=compiled_contract['abi'], bytecode=compiled_contract['bin'])
    
    # contructor is called and contract is deployed; transaction hash created; used as identifier
    tx_hash = contract_interface.constructor(*c_args).transact()
    
    # Wait for the transaction to be mined and placed into a block; provide tx_hash; get a receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    # Get the deployed contract instance from the blockchain with the newly-deployed contract's address
    deployed_contract_interface = w3.eth.contract(
        address=tx_receipt.contractAddress,
        abi=compiled_contract['abi'],
        bytecode=compiled_contract['bin']
    )
    
    return tx_receipt, deployed_contract_interface

In [ ]:
# look at accounts provided by eth-tester
TESTER.get_accounts()

In [ ]:
# set up accounts for each party
sensor_account = TESTER.get_accounts()[0]
supplier_account = TESTER.get_accounts()[1]
carrier_account = TESTER.get_accounts()[2]
fulfillment_center_account = TESTER.get_accounts()[3]
pool_point_account = TESTER.get_accounts()[4]
delivery_agent_account = TESTER.get_accounts()[5]

In [ ]:
# compile the supply damage chain contract; gets the compiled code
sc_compiled = compile_contract("SupplyDamageChain")

In [ ]:
# set up the provider which is the connection to the blockchain
provider = Web3.EthereumTesterProvider(ethereum_tester=TESTER)

In [ ]:
# get the web3 instance and the sensor account
# returns the web3 object which connects with the blockchain
sc_w3 = get_w3(provider, sensor_account)

In [ ]:
# deploy SupplyDamageChain contract by passing in connection and the compiled code
# returns the receipt for the contract deployment transaction and the deployed contract's interface
sc_receipt, damage_contract = deploy_contract(sc_w3, sc_compiled)

In [ ]:
# see the functions in the contract
damage_contract.all_functions()

In [ ]:
# helpfer function but no execution
def exec_call(contract_interface, function_name, *f_args):
    func_inst = contract_interface.get_function_by_name(function_name)

    return_value = func_inst(*f_args).call()
    return return_value

In [ ]:
# helpfer function for executing a transaction and return the transaction receipt
def exec_transact_receipt(w3, contract_interface, function_name, *f_args):
    func_inst = contract_interface.get_function_by_name(function_name)
    
    # get the return value first, without executing transaction
    return_value = exec_call(contract_interface, function_name, *f_args)
    
    # execute the transaction
    tx_hash = func_inst(*f_args).transact()
    # receipt does not contain values returned by function
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    return return_value, tx_receipt

In [ ]:
# execute transaction, but ignore the receipt
def exec_transact(w3, contract_interface, function_name, *f_args):
    rv, _ = exec_transact_receipt(w3, contract_interface, function_name, *f_args)
    return rv

In [ ]:
# When a customer places an order on the E-Commerce website,
# the supplier logs the product onto the blockchain using register_SupplierPartNumber
# The address of the carrier, potential fulfillment center, potential pool point, and potential delivery agent
# and customer address are passed in
# The id of the supplier part number is returned
# the contract keeps track of the latest id using the latest_id variable
# the registered supplier part number is given latest_id as the id
exec_call(damage_contract, "latest_id")

In [ ]:
# registering customer orders
# register_SupplierPartNumber takes in an account number from assigned carrier, an account number from the
# potential fulfillment center, pool point, delivery agent, and customer
supplierPartNumber1_id, supplierPartNumber1_receipt = exec_transact_receipt(sc_w3, damage_contract, "register_SupplierPartNumber", carrier_account, fulfillment_center_account, pool_point_account, delivery_agent_account, customer_account)
supplierPartNumber2_id = exec_transact(sc_w3, damage_contract, "register_SupplierPartNumber", carrier_account, fulfillment_center_account, pool_point_account, delivery_agent_account, customer_account)

print("SupplierPartNumber id: ", supplierPartNumber1_id)
print("SupplierPartNumber 2 id: ", supplierPartNumber2_id)

In [ ]:
# see gas of transaction
supplierPartNumber1_receipt

In [ ]:
# carrier arrives at Supplier's warehouse to pick up the product
# the loading dock oracles check the products for damage
# compile and deploy the DamageOracle contract; pass in the sensor account as a constructor argument for the contract
damage_oracle_compiled = compile_contract("DamageOracle")

# pass in the sensor account; deploy DamageOracle contract
damage_oracle_receipt, damage_oracle_contract = deploy_contract(sc_w3, damage_oracle_compiled, sensor_account)

In [ ]:
# oracle senses damage in a product, reports the damage as an event by calling DamageOracle.Constructor()
# passing in the product id; we get the transaction receipt
# sensor detects damage of product with id of 0 and reports the damage; emmits isDamaged event
_, damage_update_receipt_1 = exec_transact_receipt(sc_w3, damage_oracle_contract, "isDamaged", supplierPartNumber1_id)


In [ ]:
# service is set up to listen to reports of damage
# pass in receipt of the damage event; extract out the args dictionary
def get_damage_event(damage_update_receipt):
     return damage_oracle_contract.events.isDamaged().processReceipt(damage_update_receipt)[0]['args']

In [ ]:
# service logs this event along with a timestamp 
damage_event_1 = get_damage_event(damage_update_receipt_1)
damage_event_1

In [ ]:
#SupplyDamageChain.report_damage is used to report the damage to product id 0
exec_transact(sc_w3, damage_contract, "report_damage", SupplierPartNumber1_id)

In [ ]:
# the carrier takes receipt of the second item which is not damaged
# the carrier calls take receipt passing in the id of the product
# take receipt of the sku
def take_receipt(w3, contract, receipt_account, SupplierPartNumber_id):
    current_account = w3.eth.defaultAccount
    w3.eth.defaultAccount = receipt_account
    # take_receipt takes in the id of the product to take receipt of
    rv = exec_transact(w3, contract, "take_receipt", SupplierPartNumber_id)
    w3.eth.defaultAccount = current_account
    return rv

In [ ]:
# returns the string "carrier" which means the carrier has picked up the product
take_receipt(sc_w3, damage_contract, carrier_account, SupplierPartNumber2_id)

In [ ]:
# the carrier reaches the pool point in this case and the oracles in the pool point check for damaged products
# oracle does not sense damage in any other products coming off the carrier's trucks
# For the products the pool point does take receipt of, a transaction is logged
# the pool points account is passed in as an argument
# status of "pool point" is returned
take_receipt(sc_w3, damage_contract, pool_point_account, supplierPartNumber2_id)

In [ ]:
# Since product ID 1 was damaged coming out of the Supplier, the pool point cannot take receipt
# An exception is thrown if the pool point attempts to log a transaction for that product
try:
    take_receipt(sc_w3, damage_contract, pool_point_account, supplierPartNumber1_id)
except:
    print("Could not take receipt of Product %d" % supplierPartNumber1_id)